In [ ]:
import pygeos as pyg
import geopandas as gpd
from networkx.algorithms.centrality import current_flow_betweenness_centrality_subset
from structure import get_road_closures
from structure_nodes import get_junction_closures
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
import folium
import configparser
import project_paths as paths

In [ ]:
roads_path = r"..\tests\data\SU_RoadLink.shp"
roads = gpd.read_file(roads_path)
roads = roads.explode().reset_index(drop=True)
road_lines = pyg.from_shapely(roads.geometry)

In [ ]:
centre = (442000, 113000)
distance = 200
distance_max = 300
wider_factor = 1

In [ ]:
# NODES

removals, candidates = get_junction_closures(road_lines, centre, distance, distance_max, wider_factor)
# pyg.prepare(candidates)
# intersection1 = np.array([pyg.intersects(candidates, edge) for edge in pyg.get_point(removals, 0)])
# intersection2 = np.array([pyg.intersects(candidates, edge) for edge in pyg.get_point(removals, -1)])
# intersection = np.max(intersection1 * intersection2, axis=0)

In [ ]:
mls_road_lines = pyg.multilinestrings(road_lines)
display_roads = pyg.clip_by_rect(mls_road_lines, *pyg.bounds(pyg.buffer(pyg.points(*centre), distance_max + 500)))
geometry_gdf = gpd.GeoDataFrame({"geometry": [display_roads,
                                              pyg.multipolygons(pyg.buffer(removals, 5)),
                                              pyg.points(*centre),
                                              pyg.buffer(pyg.points(*centre), distance),
                                              pyg.buffer(pyg.points(*centre), distance_max)]},
                                crs="EPSG:27700").to_crs("EPSG:4326")

In [ ]:
config = configparser.ConfigParser()
config.read(paths.config_path)
API_KEY = config['KEYS']['API_KEY']
MAP_CENTRE = [geometry_gdf.geometry[2].y, geometry_gdf.geometry[2].x]
LAYER = "Light_3857"
TILES = f"https://api.os.uk/maps/raster/v1/zxy/{LAYER}/{{z}}/{{x}}/{{y}}.png?key={API_KEY}"
ATTR = f"Contains OS data © Crown copyright and database right {datetime.now().year}"

folium_params = {"location": MAP_CENTRE,
                 "zoom_start": 15,
                 "tiles": TILES,
                 "attr": ATTR}

os.environ['PROJ_NETWORK'] = 'OFF'

def get_geojson_map(gdf):
    m = folium.Map(**folium_params)
    folium.GeoJson(gdf.geometry[0],
                   style_function=lambda x: {
#                    'fillColor': "yellow",
                   'color': 'black',
#                    'fillOpacity': 0.4,
#                    'lineOpacity': 0.9,
                   'weight': 0.4}).add_to(m)
    folium.GeoJson(gdf.geometry[1],
                   style_function=lambda x: {
#                    'fillColor': "green",
                   'color': 'red',
#                    'fillOpacity': 0.6,
                   'lineOpacity': 1,
                   'weight': 2}).add_to(m)
    folium.GeoJson(gdf.geometry[3],
                   style_function=lambda x: {
#                    'fillColor': "green",
                   'color': 'red',
                   'fillOpacity': 0.4,
                   'lineOpacity': 0.5,
                   'weight': 0.5}).add_to(m)
    folium.GeoJson(gdf.geometry[4],
                   style_function=lambda x: {
#                    'fillColor': "green",
                   'color': 'red',
                   'fillOpacity': 0.1,
                   'lineOpacity': 0.3,
                   'weight': 0.5}).add_to(m)
    folium.Marker(MAP_CENTRE,
                  icon=folium.Icon(color="red", icon="glyphicon-exclamation-sign")
                   ).add_to(m)
    return m

In [ ]:
get_geojson_map(geometry_gdf)

In [ ]:
# EDGES

# removals, candidates = get_road_closures(road_lines, centre, distance, distance_max, wider_factor)
# pyg.prepare(candidates)
# intersection1 = np.array([pyg.intersects(candidates, edge) for edge in pyg.get_point(removals, 0)])
# intersection2 = np.array([pyg.intersects(candidates, edge) for edge in pyg.get_point(removals, -1)])
# intersection = np.max(intersection1 * intersection2, axis=0)

In [ ]:
# geometry_gdf = gpd.GeoDataFrame({"geometry": [pyg.multilinestrings(road_lines),
#                                               pyg.multilinestrings(candidates[intersection]),
#                                               pyg.points(*centre),
#                                               pyg.buffer(pyg.points(*centre), distance)]},
#                                 crs="EPSG:27700").to_crs("EPSG:4326")